# Python Exercise 4: Basic Ad Analyses
Hathaway Zhang <br>
104369396  <br>
Sep.18, 2018  <br>

In [2]:
# Import necessary package for the assignment 
import pandas as pd
from pandas import DataFrame
import numpy as np
import statsmodels.api as sm

In [3]:
# load data and preview
dfRaw = pd.read_csv("Travel Pony Facebook.csv")
dfRaw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3705 entries, 0 to 3704
Data columns (total 18 columns):
Start Date                        3705 non-null object
End Date                          3705 non-null object
Campaign Name                     3705 non-null object
Reach                             3705 non-null int64
Frequency                         3705 non-null float64
Impressions                       3705 non-null int64
Clicks                            3705 non-null int64
Unique Clicks                     3705 non-null int64
Amount Spent (USD)                3705 non-null float64
Page Likes                        3705 non-null int64
Page Engagement                   3705 non-null int64
Post Engagement                   3705 non-null int64
Post Likes                        3705 non-null int64
Post Comments                     3705 non-null int64
Post Shares                       3705 non-null int64
Photo Views                       3705 non-null int64
Website Clicks        

### 1. Create the analytics 'cost per impression'
What day of the week works best? (What day is it cheapest to generate impressions) 
What day works worst? (What day is most expensive)

In [4]:
# create new column
dfRaw['Cost Per Impression'] = dfRaw['Amount Spent (USD)'] / dfRaw['Impressions']
# format start date into day name
dfRaw['Start Date'] = pd.to_datetime(dfRaw['Start Date'],errors='coerce',format='%m/%d/%y')
dfRaw['Day'] = dfRaw['Start Date'].dt.dayofweek
days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
dfRaw.insert(0,'Day Of Week',dfRaw['Day'].apply(lambda x: days[x]))
dfRaw.head()

# calculate cost per impression and find the best and worst day
dfR1 = pd.DataFrame(dfRaw.groupby(['Day Of Week']).mean()['Cost Per Impression'].sort_values(ascending=False)).reset_index()
print(dfR1['Day Of Week'][6],"works best.")
dfR2 = pd.DataFrame(dfRaw.groupby(['Day Of Week']).mean()['Cost Per Impression'].sort_values(ascending=True)).reset_index()
print(dfR2['Day Of Week'][6],"works worst.")

Saturday works best.
Friday works worst.


### 2. Compute Correlation

In [5]:
# compute the correlations
print(dfRaw[['Amount Spent (USD)','Reach']].corr())
print(dfRaw[['Amount Spent (USD)','Frequency']].corr())
print(dfRaw[['Amount Spent (USD)','Unique Clicks']].corr())
print(dfRaw[['Amount Spent (USD)','Page Likes']].corr())
print("Correlation between Amount Spent and Unique Clicks is the stronges, which means unique clicks is highly associated with the amount spent and unique click is more correlated with unique clicks comparing with reach, frequency, and page likes.")

                    Amount Spent (USD)     Reach
Amount Spent (USD)            1.000000  0.703124
Reach                         0.703124  1.000000
                    Amount Spent (USD)  Frequency
Amount Spent (USD)            1.000000   0.130201
Frequency                     0.130201   1.000000
                    Amount Spent (USD)  Unique Clicks
Amount Spent (USD)            1.000000       0.882993
Unique Clicks                 0.882993       1.000000
                    Amount Spent (USD)  Page Likes
Amount Spent (USD)            1.000000    0.757612
Page Likes                    0.757612    1.000000
Correlation between Amount Spent and Unique Clicks is the stronges, which means unique clicks is highly associated with the amount spent and unique click is more correlated with unique clicks comparing with reach, frequency, and page likes.


### 3. Simple Multiple Regression Analysis

In [8]:
X = dfRaw[["Reach","Frequency"]]
y = dfRaw["Unique Clicks"]
# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Unique Clicks   R-squared:                       0.557
Model:                            OLS   Adj. R-squared:                  0.556
Method:                 Least Squares   F-statistic:                     2325.
Date:                Tue, 16 Oct 2018   Prob (F-statistic):               0.00
Time:                        16:03:08   Log-Likelihood:                -15973.
No. Observations:                3705   AIC:                         3.195e+04
Df Residuals:                    3703   BIC:                         3.196e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Reach          0.0019   3.12e-05     62.490      0.000       0.002       0.002
Frequency      3.6139      0.298     12.109      0.000       3.029       4.199
==============================================================================
Omnibus:                     5107.616   Durbin-Watson:                   0.803
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5128392.747
Skew:                           7.331   Prob(JB):                         0.00
Kurtosis:                     184.674   Cond. No.                     9.84e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.84e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""